In [1]:
import numpy as np
import math

In [2]:
train_data = np.zeros((200,1000))
file_name = 'test20.txt'
outpu_file_name = 'newCustom20.txt'
def loading_train_data():
    training_file = open('train.txt', 'r')
    row = training_file.readline()
    for i in range(0,200):
        row = row.split()
        row_list = []
        for j in row:
            row_list.append(int(j))
        for k in range(0,1000):
            train_data[i][k] = row_list[k]
        row = training_file.readline()
    training_file.close()
    return train_data

In [3]:
loading_train_data()

array([[ 5.,  3.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  4.,  0., ...,  0.,  0.,  0.]])

In [4]:
print (train_data[0][0:10])

[ 5.  3.  0.  3.  3.  5.  0.  1.  5.  3.]


In [5]:
predict_rating_list = {}
def get_predict_movie_list():
    test_data = open(file_name,'r')
    row = test_data.readline()
    while row:
        row = row.split()
        each_row = []
        for i in row:
            each_row.append(int(i))
        if each_row[2] == 0:
            if each_row[0] in predict_rating_list:
                temp = predict_rating_list[each_row[0]]
                temp.append(each_row[1])
                predict_rating_list.update({each_row[0]:temp})
            else:
                new_list = []
                new_list.append(each_row[1])
                predict_rating_list.update({each_row[0]:new_list}) 
        row = test_data.readline()
    test_data.close()
    return predict_rating_list  

In [6]:
predict_userid_to_moveid_list = {}
predict_userid_to_ratingid_list = {}
def get_movie_rating_list_test():
    test_data = open(file_name,'r')
    row = test_data.readline()
    while row:
        row = row.split()
        each_row = []
        for i in row:
            each_row.append(int(i))
        if each_row[2] != 0:
            if each_row[0] in predict_userid_to_moveid_list:
                temp_movie = predict_userid_to_moveid_list[each_row[0]]
                temp_rating = predict_userid_to_ratingid_list[each_row[0]]
                temp_movie.append(each_row[1])
                temp_rating.append(each_row[2])
                predict_userid_to_moveid_list.update({each_row[0]:temp_movie})
                predict_userid_to_ratingid_list.update({each_row[0]:temp_rating})
            else:
                new_list = []
                new_list1 = []
                new_list.append(each_row[1])
                new_list1.append(each_row[2])
                predict_userid_to_moveid_list.update({each_row[0]:new_list})
                predict_userid_to_ratingid_list.update({each_row[0]:new_list1})
        row = test_data.readline()
    test_data.close()
    return

In [7]:
get_movie_rating_list_test()
# print (predict_userid_to_moveid_list[201])
# print (predict_userid_to_ratingid_list[201])

In [8]:
user_avgerage_rating_training_map = {}
def user_avgerage_rating_training():
    for user in range(0,200):
        count = 0
        total = 0.0
        for movie_id in range(0,1000):
            if train_data[user][movie_id] != 0:
                total += train_data[user][movie_id]*1.0
                count += 1
            
        average_rating = total/count
        user_avgerage_rating_training_map[user + 1] = average_rating
    return 

In [9]:
def inverse_user_freq(iuf_movie_id):
    m = 0
    for user in range(0,200):
        if train_data[user][iuf_movie_id-1] != 0:
            m += 1
    if m != 0:
        IUF = math.log10(200.0/m)
    else:
        IUF = 1
    return IUF

In [10]:
item_user_movie_list = {}
def item_based_adjusted_coSign(user_id, movie_id, k_nb):
    similar_neighbor_list = []
    movie_list = predict_userid_to_moveid_list[user_id]
    
    for i in range(len(movie_list)):
        rated_m = movie_list[i]
        num = 0.0
        d_i = 0.0
        d_j = 0.0
        
        common_user = 0
        temp_list = {}
        for user in range(0,200):
            average_rating_train = user_avgerage_rating_training_map[user + 1]
            if(train_data[user][rated_m] != 0) & (train_data[user][movie_id-1] != 0):
                num += (train_data[user][rated_m - 1] - average_rating_train) * (train_data[user][movie_id-1] -average_rating_train)
                d_i += math.pow((train_data[user][rated_m-1]),2)
                d_j += math.pow((train_data[user][movie_id-1]),2)
                common_user += 1
        
        if common_user > 1:
            d_total = math.sqrt(d_i)*math.sqrt(d_j)
            item_similarity = num/d_total
            
            item_similarity *= (common_user/(common_user+2))
            
            item_similarity *= math.pow(math.fabs(item_similarity),1.5)
            
            temp_list['userid'] = i + 1
            temp_list['similarity'] = item_similarity
            similar_neighbor_list.append(temp_list)
            item_user_movie_list[i +1] = movie_id
        
    similar_neighbor_list.sort(key=lambda x:x['similarity'], reverse=True)
    k_similar_neighbor_list = []
        
    for i in range(0, len(similar_neighbor_list)):
        if i < k_nb:
            k_similar_neighbor_list.append(similar_neighbor_list[i])
    return k_similar_neighbor_list
                                
    

In [11]:
def caluculate_item_based(user_id,movie_id,k_nb):
    k_similarity_list = item_based_adjusted_coSign(user_id,movie_id,k_nb)
    
    average_user_rating = avg_rating_same_user(user_id)
    
    average_movie_rating = average_rating_for_each_movie_map[movie_id]
    
    num = 0.0
    d = 0.0
    
    movie_l = predict_userid_to_moveid_list[user_id]
    rating_l = predict_userid_to_ratingid_list[user_id]
    
#     print("movie_l",movie_l)
#     print("rating_l",rating_l)
    
    for i in range(0 , len(movie_l)):
        m_id = movie_l[i]
        rating = rating_l[i]
        
        for j in range(0, len(k_similarity_list)):
            curr = k_similarity_list[j]
            user_id = curr['userid']
            item_similarity = curr['similarity']
            similar_movie_id = item_user_movie_list[user_id]
            
            if m_id == similar_movie_id:
                num += similarity * rating
                d += math.fabs(item_similarity)
        
    if d != 0.0:
        res = num/d
    elif average_movie_rating != 0:
        res = average_movie_rating
    else:
        res = average_user_rating
    
    if math.isnan(res):
        res = int(round(1))
    else:
        res = int(round(res))
        
    if res > 5:
        res = 5
    elif res <= 0:
        res = 1
        
    return res
        

In [12]:
def calculate_peasrson_correlation(user_id,k_nb):
    similar_neighbor_list = []
    
    average_rating_testdata = avg_rating_same_user(user_id)
    
    movie_l = predict_userid_to_moveid_list[user_id]
    rating_l = predict_userid_to_ratingid_list[user_id]
    
    for user in range(0,200):
        num = 0.0
        d_test = 0.0
        d_train = 0.0
        similarity_map = {}
        
        iuf_movie_id = 0
        common_movie = 0
        for movie in range(0, len(movie_l)):
            movie_id = movie_l[movie]
            user_rating = rating_l[movie]
            train_data_rating = train_data[user][movie_id - 1]
            iuf_movie_id = movie_id
            
            if(user_rating != 0) & (train_data_rating != 0):
                average_rating_from_train = user_avgerage_rating_training_map[user+1]
                num += (user_rating - average_rating_testdata) * (train_data_rating - average_rating_from_train)
                d_test += math.pow((user_rating - average_rating_testdata),2)
                d_train += math.pow((train_data_rating - average_rating_from_train),2)
                common_movie += 1
        
        d_total = 0.0
        if(d_test > 0) & (d_train > 0):
            d_total = math.sqrt(d_test)*math.sqrt(d_train)
        
        if d_train != 0.0:
            pearson_similarity = num/d_total
            
#             #Dirichlet smoothing
#             pearson_similarity *= (common_movie/(common_movie + 2))
            
            #case amplification
            pearson_similarity *= math.pow(math.fabs(pearson_similarity), 1.5)
            
            #IUF
            IUF = inverse_user_freq(iuf_movie_id)
            pearson_similarity *= IUF
            
            similarity_map['userid'] = user + 1
            similarity_map['similarity'] = pearson_similarity
            similar_neighbor_list.append(similarity_map)
    
    similar_neighbor_list.sort(key=lambda x:x['similarity'], reverse=True)
    k_similar_neighbor_list = []
    for i in range(0, len(similar_neighbor_list)):
        if i < k_nb:
            k_similar_neighbor_list.append(similar_neighbor_list[i])
    return k_similar_neighbor_list

In [13]:
def rating_from_pearson(user_id,movie_id,k_nb):
    k_similar_neighbor_list = calculate_peasrson_correlation(user_id,k_nb)
    
    average_rating_testdata = avg_rating_same_user(user_id)
    
    average_each_movie_rating = average_rating_for_each_movie_map[movie_id]
    
    num = 0.0
    d = 0.0
    
    for i in range(0, len(k_similar_neighbor_list)):
        current_neighbor = k_similar_neighbor_list[i]
        train_user_id = current_neighbor['userid']
        
        average_train_rating = user_avgerage_rating_training_map[train_user_id]
        
        if train_data[train_user_id-1][movie_id-1] > 0:
            weighted = train_data[train_user_id-1][movie_id-1] - average_train_rating
            num += weighted * current_neighbor['similarity']
            d += math.fabs(current_neighbor['similarity'])
            
    if d != 0:
        result = average_rating_testdata + num/d
    elif average_each_movie_rating != 0:
        result = average_each_movie_rating
    else:
        result = average_rating_testdata
    
    if math.isnan(result):
        result = int(round(1))
    else:
        result = int(round(result))
    result = int(round(result))
    if result >= 5:
        result = 5
    elif result <= 0:
        result = 1
    
    return result;

In [14]:
average_rating_for_each_movie_map = {}
def average_rating_for_each_movie():
    for movie_id in range(0,1000):
        total = 0.0
        count = 0
        for user_id in range(0,200):
            if train_data[user_id][movie_id] != 0:
                total += train_data[user_id][movie_id]
                count += 1
        
        if count > 0:
            average_rating = total/count
            average_rating_for_each_movie_map[movie_id + 1] = average_rating
        else:
            average_rating_for_each_movie_map[movie_id + 1] = 0.0
    return

In [15]:
def calculate_cosine_similarity(user_id,k_nb):
    neighbor_list = []
    
    movie_l = predict_userid_to_moveid_list[user_id]
    rating_l = predict_userid_to_ratingid_list[user_id]
    
    for user in range(0,200):
        num = 0.0
        d_test = 0.0
        d_train = 0.0
        similarity_map = {}
        for m in range(0,len(movie_l)):
            movie_id = movie_l[m]
            
            if((rating_l[m] != 0) & (train_data[user][movie_id-1] != 0)):
                num += rating_l[m] * train_data[user][movie_id-1]
                d_test += math.pow(rating_l[m],2)
                d_train += math.pow(train_data[user][movie_id-1],2)
                
        d_total = math.sqrt(d_test) * math.sqrt(d_train)
        
        if d_total != 0.0:
            cosine_simlarity = num/d_total
            similarity_map['userid'] = user + 1
            similarity_map['similarity'] = cosine_simlarity
            neighbor_list.append(similarity_map)
    
    neighbor_list.sort(key=lambda x:x['similarity'], reverse=True)
    k_neighbor_list = []
    for i in range(0, len(neighbor_list)):
        if i < k_nb:
            k_neighbor_list.append(neighbor_list[i])
    return k_neighbor_list

In [16]:

avg_rating_same_movie_map = {}
def avg_rating_same_movie():
    
    for movie in range(0,1000):
        sum1 = 0.0
        count = 0
        
        for user in range(0,200):
            if train_data[user][movie] != 0:
                sum1 += train_data[user][movie]
                count += 1
        
        if count > 0:
            avg = sum1/count
            avg_rating_same_movie_map[movie + 1] = avg
        else:
            avg_rating_same_movie_map[movie + 1] = 0.0
        
    return

In [17]:
def avg_rating_same_user(user_id):
    current_user_list = predict_userid_to_ratingid_list[user_id]
    t_rating = 0.0
    
    for i in range(0,len(current_user_list)):
        t_rating += current_user_list[i]
    
    average_rating = t_rating/len(current_user_list)
    return average_rating
    

In [18]:
def calculate_rating_cosine(user_id,movie_id,k_nb):
    user_similarity_list = calculate_cosine_similarity(user_id,k_nb)
    
    num = 0.0
    d = 0.0
    
    average_user_rating = avg_rating_same_user(user_id)
    
    for i in range(0, len(user_similarity_list)):
        temp = user_similarity_list[i]
        user_id_train = temp['userid']
        
        if train_data[user_id_train - 1][movie_id - 1] > 0:
            num += temp['similarity'] * train_data[user_id_train - 1][movie_id - 1]
            d += temp['similarity']
        
    if(d != 0):
        result = num/d
    elif avg_rating_same_movie_map[movie_id]:
        result = avg_rating_same_movie_map[movie_id]
    else:
        result = average_user_rating
        
    result = int(round(result))
    return result
            

In [19]:
def main():
    k_nb = 60
    get_predict_movie_list()
    avg_rating_same_movie()
    user_avgerage_rating_training() # for pearson
    average_rating_for_each_movie() # for pearson
    result_file = open(outpu_file_name , 'w')
    for user_id in predict_rating_list:
        for movie_id in predict_rating_list[user_id]:
            rating_of_pearson =  rating_from_pearson(user_id,movie_id,k_nb)
            rating_of_item_based = caluculate_item_based(user_id,movie_id,k_nb)
            res = rating_of_pearson * 0.3 + rating_of_item_based * 0.7
            if res <= 0:
                res = 1
            elif res >= 5:
                res = 5
            row = str(user_id) + " " + str(movie_id) + " " + str(res)
            print(user_id,movie_id,rating_of_item_based)
            result_file.write(row + '\n')
            print(user_id,"Done")
    result_file.close()
    return print("Finished")
    

In [ ]:
main()

/Users/balajisai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in double_scalars


401 2 3
401 Done
401 4 4
401 Done
401 8 4
401 Done
401 9 4
401 Done
401 11 4
401 Done
401 17 3
401 Done
401 20 3
401 Done
401 23 4
401 Done
401 26 4
401 Done
401 27 3
401 Done
401 31 4
401 Done
401 32 4
401 Done
401 36 2
401 Done
401 37 2
401 Done
401 39 3
401 Done
401 42 4
401 Done
401 46 4
401 Done
401 47 4
401 Done
401 48 4
401 Done
401 50 4
401 Done
401 51 3
401 Done
401 53 3
401 Done
401 56 4
401 Done
401 57 4
401 Done
401 58 4
401 Done
401 59 4
401 Done
401 61 4
401 Done
401 62 3
401 Done
401 64 5
401 Done
401 68 3
401 Done
401 69 4
401 Done
401 70 4
401 Done
401 71 4
401 Done
401 76 4
401 Done
401 77 3
401 Done
401 79 4
401 Done
401 81 4
401 Done
401 82 4
401 Done
401 86 4
401 Done
401 87 4
401 Done
401 89 4
401 Done
401 92 4
401 Done
401 93 4
401 Done
401 95 4
401 Done
401 97 4
401 Done
401 98 4
401 Done
401 99 4
401 Done
401 100 4
401 Done
401 117 4
401 Done
401 118 3
401 Done
401 121 4
401 Done
401 123 3
401 Done
401 124 4
401 Done
401 127 4
401 Done
401 128 3
401 Done
401 12

405 Done
405 866 3
405 Done
405 871 3
405 Done
405 875 3
405 Done
405 978 2
405 Done
405 986 2
405 Done
405 993 3
405 Done
405 997 2
405 Done
406 56 4
406 Done
406 70 4
406 Done
406 97 4
406 Done
406 194 4
406 Done
406 202 4
406 Done
406 204 4
406 Done
406 216 4
406 Done
406 310 4
406 Done
406 662 4
406 Done
406 663 4
406 Done
406 996 2
406 Done
407 1 4
407 Done
407 9 4
407 Done
407 14 4
407 Done
407 50 4
407 Done
407 271 3
407 Done
407 286 4
407 Done
408 4 4
408 Done
408 15 4
408 Done
408 23 4
408 Done
408 40 3
408 Done
408 44 4
408 Done
408 49 3
408 Done
408 50 4
408 Done
408 56 4
408 Done
408 58 4
408 Done
408 65 4
408 Done
408 72 3
408 Done
408 73 3
408 Done
408 88 4
408 Done
408 94 3
408 Done
408 96 4
408 Done
408 99 4
408 Done
408 105 2
408 Done
408 127 4
408 Done
408 134 4
408 Done
408 135 4
408 Done
408 152 3
408 Done
408 154 4
408 Done
408 160 4
408 Done
408 161 4
408 Done
408 167 3
408 Done
408 172 4
408 Done
408 179 4
408 Done
408 180 4
408 Done
408 182 4
408 Done
408 185 4


415 576 3
415 Done
415 578 3
415 Done
415 586 3
415 Done
415 597 3
415 Done
415 665 3
415 Done
415 679 3
415 Done
415 685 4
415 Done
415 779 3
415 Done
416 4 4
416 Done
416 8 4
416 Done
416 33 4
416 Done
416 39 3
416 Done
416 47 4
416 Done
416 55 4
416 Done
416 168 4
416 Done
416 186 4
416 Done
416 203 4
416 Done
416 209 4
416 Done
416 265 4
416 Done
416 273 4
416 Done
416 294 3
416 Done
416 410 3
416 Done
416 430 4
416 Done
416 504 4
416 Done
416 516 4
416 Done
416 591 4
416 Done
416 654 4
416 Done
416 659 4
416 Done
416 762 3
416 Done
417 13 4
417 Done
417 215 4
417 Done
417 223 4
417 Done
417 258 4
417 Done
417 269 4
417 Done
418 3 3
418 Done
418 4 4
418 Done
418 7 4
418 Done
418 17 3
418 Done
418 23 4
418 Done
418 29 3
418 Done
418 32 4
418 Done
418 39 3
418 Done
418 48 4
418 Done
418 50 4
418 Done
418 53 3
418 Done
418 55 4
418 Done
418 59 4
418 Done
418 69 4
418 Done
418 79 4
418 Done
418 92 4
418 Done
418 94 3
418 Done
418 96 4
418 Done
418 100 4
418 Done
418 108 3
418 Done
418 

/Users/balajisai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


...

453 379 3
453 Done
453 402 4
453 Done
453 441 2
453 Done
453 443 4
453 Done
453 466 4
453 Done
453 471 4
453 Done
453 509 4
453 Done
453 531 4
453 Done
453 535 4
453 Done
453 546 3
453 Done
453 553 3
453 Done
453 558 4
453 Done
453 559 3
453 Done
453 574 4
453 Done
453 581 4
453 Done
453 582 4
453 Done
453 583 3
453 Done
453 596 3
453 Done
453 603 4
453 Done
453 671 4
453 Done
453 684 4
453 Done
453 703 4
453 Done
453 713 4
453 Done
453 714 4
453 Done
453 716 4
453 Done
453 722 3
453 Done
453 727 4
453 Done
453 736 4
453 Done
453 739 4
453 Done
453 740 3
453 Done
453 741 3
453 Done
453 742 4
453 Done
453 747 4
453 Done
453 778 4
453 Done
453 781 4
453 Done
453 794 3
453 Done
453 800 3
453 Done
453 815 3
453 Done
453 860 2
453 Done
453 869 3
453 Done
453 872 2
453 Done
453 928 3
453 Done
453 943 3
453 Done
454 1 4
454 Done
454 2 3
454 Done
454 4 4
454 Done
454 8 4
454 Done
454 11 4
454 Done
454 12 4
454 Done
454 13 4
454 Done
454 15 4
454 Done
454 22 4
454 Done
454 31 4
454 Done
454 38 

458 156 4
458 Done
458 157 4
458 Done
458 158 3
458 Done
458 159 4
458 Done
458 160 4
458 Done
458 164 4
458 Done
458 168 4
458 Done
458 169 4
458 Done
458 173 4
458 Done
458 174 4
458 Done
458 176 4
458 Done
458 181 4
458 Done
458 183 4
458 Done
458 184 3
458 Done
458 185 4
458 Done
458 186 4
458 Done
458 188 4
458 Done
458 189 4
458 Done
458 192 4
458 Done
458 193 4
458 Done
458 195 4
458 Done
458 196 4
458 Done
458 197 4
458 Done
458 198 4
458 Done
458 201 3
458 Done
458 202 4
458 Done
458 204 4
458 Done
458 206 3
458 Done
458 207 4
458 Done
458 209 4
458 Done
458 214 3
458 Done
458 219 3
458 Done
458 222 4
458 Done
458 223 4
458 Done
458 225 3
458 Done
458 226 4
458 Done
458 227 3
458 Done
458 228 4
458 Done
458 229 3
458 Done
458 230 3
458 Done
458 231 3
458 Done
458 232 3
458 Done
458 233 4
458 Done
458 234 4
458 Done
458 235 3
458 Done
458 238 4
458 Done
458 239 4
458 Done
458 240 3
458 Done
458 241 3
458 Done
458 246 4
458 Done
458 249 3
458 Done
458 250 3
458 Done
458 252 3
45

460 Done
460 577 3
460 Done
460 578 3
460 Done
460 591 4
460 Done
460 594 3
460 Done
460 597 3
460 Done
460 616 4
460 Done
460 624 3
460 Done
460 625 3
460 Done
460 636 3
460 Done
460 638 4
460 Done
460 649 4
460 Done
460 652 4
460 Done
460 654 4
460 Done
460 659 4
460 Done
460 660 4
460 Done
460 662 4
460 Done
460 663 4
460 Done
460 666 4
460 Done
460 671 4
460 Done
460 679 3
460 Done
460 684 4
460 Done
460 685 4
460 Done
460 702 3
460 Done
460 709 4
460 Done
460 710 3
460 Done
460 712 3
460 Done
460 719 2
460 Done
460 721 3
460 Done
460 725 3
460 Done
460 727 4
460 Done
460 728 3
460 Done
460 732 4
460 Done
460 739 4
460 Done
460 740 3
460 Done
460 741 3
460 Done
460 746 4
460 Done
460 751 3
460 Done
460 753 4
460 Done
460 760 2
460 Done
460 762 3
460 Done
460 763 3
460 Done
460 764 3
460 Done
460 778 4
460 Done
460 781 4
460 Done
460 789 4
460 Done
460 792 4
460 Done
460 797 3
460 Done
460 802 3
460 Done
460 804 2
460 Done
460 809 3
460 Done
460 810 3
460 Done
460 820 2
460 Done
460

464 Done
464 988 2
464 Done
464 993 3
464 Done
465 4 4
465 Done
465 9 4
465 Done
465 11 4
465 Done
465 39 3
465 Done
465 56 4
465 Done
465 92 4
465 Done
465 98 4
465 Done
465 111 4
465 Done
465 117 4
465 Done
465 121 4
465 Done
465 156 4
465 Done
465 208 4
465 Done
465 235 3
465 Done
465 240 3
465 Done
465 250 3
465 Done
465 252 3
465 Done
465 268 4
465 Done
465 276 4
465 Done
465 301 3
465 Done
465 313 4
465 Done
465 327 3
465 Done
465 340 4
465 Done
465 346 3
465 Done
465 426 4
465 Done
465 476 3
465 Done
465 546 3
465 Done
465 652 4
465 Done
465 682 2
465 Done
465 710 3
465 Done
465 815 3
465 Done
465 845 4
465 Done
465 926 2
465 Done
465 952 3
465 Done
466 12 4
466 Done
466 13 4
466 Done
466 22 4
466 Done
466 50 4
466 Done
466 69 4
466 Done
466 121 4
466 Done
466 127 4
466 Done
466 132 4
466 Done
466 134 4
466 Done
466 173 4
466 Done
466 178 4
466 Done
466 182 4
466 Done
466 196 4
466 Done
466 197 4
466 Done
466 200 4
466 Done
466 202 4
466 Done
466 214 3
466 Done
466 216 4
466 Don

470 Done
470 79 4
470 Done
470 81 4
470 Done
470 82 4
470 Done
470 87 4
470 Done
470 91 3
470 Done
470 92 4
470 Done
470 99 4
470 Done
470 111 4
470 Done
470 121 4
470 Done
470 124 4
470 Done
470 125 4
470 Done
470 129 3
470 Done
470 132 4
470 Done
470 133 4
470 Done
470 134 4
470 Done
470 135 4
470 Done
470 139 3
470 Done
470 144 4
470 Done
470 147 4
470 Done
470 148 3
470 Done
470 150 4
470 Done
470 151 4
470 Done
470 152 3
470 Done
470 153 4
470 Done
470 155 3
470 Done
470 158 3
470 Done
470 159 4
470 Done
470 160 4
470 Done
470 161 4
470 Done
470 163 4
470 Done
470 164 4
470 Done
470 166 4
470 Done
470 167 3
470 Done
470 172 4
470 Done
470 174 4
470 Done
470 175 4
470 Done
470 177 4
470 Done
470 179 4
470 Done
470 180 4
470 Done
470 181 4
470 Done
470 182 4
470 Done
470 183 4
470 Done
470 186 4
470 Done
470 187 4
470 Done
470 188 4
470 Done
470 192 4
470 Done
470 193 4
470 Done
470 194 4
470 Done
470 195 4
470 Done
470 196 4
470 Done
470 198 4
470 Done
470 199 4
470 Done
470 200 4
